In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
pip install torch torchvision torchaudio


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 324, in run
    session = self.get_default_session(options)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/index_command.py", line 71, in get_default_session
    self._session = self.enter_context(self._build_session(options))
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/index_command.py", line 100, in _build_session
    session = PipSession(
              ^^^^^^^^^

In [ ]:
import pandas as pd
import numpy as np
import torch
import pathlib as Path
import json
import os

###Creating base model
Combination of all json files from QA formatting

In [ ]:
#Comnbining all json files from the QA formatting into one
import os

folder_path = "/content/json_files"
combined_data = []

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                try:
                    combined_data.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"Error decoding line in file {filename}:")
                    print("Line:", line)
                    print("Error:", e)

if combined_data:
    print("Keys in the first JSON object:", combined_data[0].keys())
else:
    print("No data loaded.")


FileNotFoundError: [Errno 2] No such file or directory: '/content/json_files'

In [ ]:
#The json from the CSV still had more columns than needed and more than the other datasets. For coherence, only the 3 relevant should be extracted
std_data = []
for record in combined_data:
    standardized_record = {
        'question': record.get('question', ''),
        'answer': record.get('answer', ''),
        'context': record.get('context', '')  # Add other common fields if needed
    }
    standardized_data.append(standardized_record)

print("Standardized record keys:", std_data[0].keys())

Standardized record keys: dict_keys(['question', 'answer', 'context'])


In [ ]:
#Check whether the combination really worked
print("Total records loaded:", len(combined_data)) #The number should be higher
print("Records from first file:", len(data))


In [ ]:
df = pd.DataFrame(std_data)
print(df.head())

                                            question  \
0                   What is the EPA's goal for 2020?   
1  What is the goal of EPA's climate adaptation p...   
2     What is the definition of extreme temperature?   
3  What are the historic All 50 states considered...   
4  What percentage of buildings are projected to ...   

                                              answer  \
0  EPA is using a number of strategies to bolster...   
1  A, B, C, D, F, H, W, E and F, and G, and (2) A...   
2  long-term economic, social, and demographic im...   
3  High, very high, or extreme risk according to ...   
4  2050 2080 2050 2080 Extreme Heat: Percent of b...   

                                             context  
0  Climate change poses significant risks to EPA’...  
1         climate adaptation both within the Agen...  
2                                         Section...  
3  Wildfire Risk   Measured as whether an asset i...  
4  2A. Climate Hazard Exposures and Impacts Affec..

Base model tokenization

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset

# Convert list to Dataset
dataset = Dataset.from_list(std_data)

In [ ]:
from transformers import T5Tokenizer

#Tokenizing function
def tokenize_function(example):
    model_inputs = tokenizer(
        example["question"],
        example["context"],
        max_length=512,
        truncation=True,
        padding="max_length")

    labels = tokenizer(
        example["answer"],
        max_length=128,
        truncation=True,
        padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)


In [ ]:
# Print the first sample to check whether everything worked well
print(tokenized_dataset[0])


{'question': 'What is the third assessment of Working Group I?', 'answer': 'The Third Assessment Report of Working Group I of the Intergovernmental Panel on Climate Change (IPCC)', 'context': 'The Third Assessment Report of Working Group I of the\nIntergovernmental Panel on Climate Change (IPCC) builds\nupon past assessments and incorporates new results from the\npast five years of research on climate change1. Many hundreds\nof scientists2 from many countries participated in its preparation\nand review.', 'input_ids': [363, 19, 8, 1025, 4193, 13, 7301, 1531, 27, 58, 1, 37, 9879, 15186, 3750, 13, 7301, 1531, 27, 13, 8, 3037, 18984, 9871, 30, 15772, 5968, 41, 4629, 2823, 61, 15326, 1286, 657, 16396, 11, 6300, 7, 126, 772, 45, 8, 657, 874, 203, 13, 585, 30, 3298, 483, 5411, 1404, 3986, 13, 7004, 357, 45, 186, 1440, 10627, 16, 165, 4537, 11, 1132, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
#Check whether input length is same as max length
sample = tokenized_dataset[0]
print("Input length:", len(sample["input_ids"]))  # Should be 512


Input length: 512


In [ ]:
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=3e-4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    predict_with_generate=True,
    report_to=[]  # Disable wandb and other logging
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# Split into 80% train, 20% eval
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


<ipython-input-13-7555ef1db4a7>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()


KeyboardInterrupt: 

In [ ]:
# Save the model and tokenizer to a directory
trainer.save_model("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")


###Testing Finetuned Model

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Finetuned_model/Finetuned_model")
tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Finetuned_model/Finetuned_model")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
conversation_history = ["User: Hi!", "Bot: Hello, how can I help you?"]
prompt = "\n".join(conversation_history)
input_text = f"context: {prompt}\nquestion: How are you?"
print(input_text)


context: User: Hi!
Bot: Hello, how can I help you?
question: How are you?


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def generate_response(user_question):
    # Automatically supply a context (e.g., retrieved from a database or previous conversation)
    default_context = "This is the relevant context from your knowledge base or conversation history."
    input_text = tokenizer.encode_plus(
        user_question,
        default_context,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    output_ids = model.generate(input_text["input_ids"], max_length=128, num_beams=5, early_stopping=True)
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response

In [ ]:
# Test input
test_question = "If everyone adopts climate policies like the US, how does the world look like in 2100?"
context = (
    "The US is one of the most important countries for climate policies"
)

# Tokenize as a pair
input_data = tokenizer.encode_plus(
    test_question,
    test_context,
    return_tensors="pt",
    max_length=512,
    truncation=True,
    padding="max_length"
)

# Generate answer with adjusted parameters
output_ids = model.generate(
    input_data["input_ids"],
    max_length=300,
    min_length=40,
    num_beams=5,
    do_sample=True,         # Enable sampling
    temperature=1.2,        # Control randomness
    no_repeat_ngram_size=3, # Avoid repeating phrases
    early_stopping=False
)

generated_answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Generated Answer:", generated_answer)


NameError: name 'test_context' is not defined

###Continued fine-tuning
The model worked well, now we want to include also the other G7 documents. Based on the model, we already have, we continue the fine-tuning.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load previously fine-tuned model from drive
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Colab Notebooks/trained_model/Finetuned_model")
tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/trained_model/Finetuned_model")

In [ ]:
#Combine all the QA formatted G7 documents
folder_path = "/content/drive/MyDrive/Colab Notebooks/qa_json_new"
new_data = []

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                try:
                    new_data.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"Error decoding line in file {filename}:")
                    print("Line:", line)
                    print("Error:", e)

if new_data:
    print("Keys in the first JSON object:", new_data[0].keys())
else:
    print("No data loaded.")

Keys in the first JSON object: dict_keys(['context', 'question', 'answer'])


In [ ]:
#Again, standardize to be sure that (only) necessary columns are preserved
standardized_data = []
for record in new_data:
    standardized_record = {
        'question': record.get('question', ''),
        'answer': record.get('answer', ''),
        'context': record.get('context', '')
    }
    standardized_data.append(standardized_record)

print("Standardized record keys:", standardized_data[0].keys())

Standardized record keys: dict_keys(['question', 'answer', 'context'])


In [ ]:
from datasets import Dataset

# Convert list to Dataset
dataset_new = Dataset.from_list(standardized_data)

In [ ]:
from transformers import T5Tokenizer

def tokenize_function(example):
    model_inputs = tokenizer(
        example["question"],
        max_length=512,
        truncation=True,
        padding="max_length")

    labels = tokenizer(
        example["answer"],
        max_length=128,
        truncation=True,
        padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset_new = dataset_new.map(tokenize_function, batched=True)

from transformers import T5Tokenizer


Map:   0%|          | 0/1746 [00:00<?, ? examples/s]

In [ ]:
#Check whether input length is same as max length
sample = tokenized_dataset_new[0]
print("Input length:", len(sample["input_ids"]))  # Should be 512


Input length: 512


In [ ]:
# Split into 80% train, 20% eval
split_dataset_new = tokenized_dataset_new.train_test_split(test_size=0.2)

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="./incremental_results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=3e-5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    predict_with_generate=True,
    report_to=[],
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset_new["train"],
    eval_dataset=split_dataset_new["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-28-b6d359622177>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,1.448661
2,No log,1.431606
3,1.472000,1.427436


TrainOutput(global_step=525, training_loss=1.4771793256487165, metrics={'train_runtime': 899.5455, 'train_samples_per_second': 4.656, 'train_steps_per_second': 0.584, 'total_flos': 2867763792052224.0, 'train_loss': 1.4771793256487165, 'epoch': 3.0})

In [ ]:
# Save the model and tokenizer
trainer.save_model("./finetuned_model_final")
tokenizer.save_pretrained("./finetuned_model_final")

('./finetuned_model_final/tokenizer_config.json',
 './finetuned_model_final/special_tokens_map.json',
 './finetuned_model_final/spiece.model',
 './finetuned_model_final/added_tokens.json')

###Implementation of Chatbot using Gradio

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Colab Notebooks/finetuned_model_final")
tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/finetuned_model_final")

def chat_response(history, user_message):
    # User message is input
    input_ids = tokenizer.encode(
        user_message,
        return_tensors="pt",
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    output_ids = model.generate(
        input_ids,
        max_length=300,
        min_length=40,
        num_beams=5,
        do_sample=True,
        temperature=1.2,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    bot_response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # Append the new turn to the history
    history.append((user_message, bot_response))
    return history, history

def clear_history():
    return []

# Define custom CSS (we tried but this doesn't seem to work)
custom_css = """
body {
    background-color: #AFE1AF !important;
}
.gradio-container .input_textbox {
    border: 2px solid #4CAF50;
    border-radius: 5px;
}
.gradio-container .output_text {
    font-size: 16px;
    color: #333;
}
"""

with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("# Hack The Stats - Climate Chatbot")
    # Display our chatbot image
    gr.Image("/content/drive/MyDrive/chatbot_pic.jpg", height=250, width=250)

    # Hold history
    chatbot = gr.Chatbot(label="Conversation History")

    # Textbox for user input
    msg = gr.Textbox(placeholder="Enter your question here...", label="Your Question")

    # Button to clear the chat history
    clear = gr.Button("Clear Chat")

    # When the user submits a message, call chat_response, which updates the history
    msg.submit(chat_response, inputs=[chatbot, msg], outputs=[chatbot, chatbot])
    clear.click(clear_history, outputs=chatbot, queue=False)

demo.launch()



/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:284: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d0fbf0617f08b5a6c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
